In [121]:
import os
import glob
import cv2
import numpy as np
from PIL import Image
import glob
import torch
import torch.nn as nn
from torchvision.utils import save_image
import matplotlib.pyplot as plt

from SemanticGuidedHumanMatting.model.model import HumanSegment, HumanMatting
import SemanticGuidedHumanMatting.utils as utils
import SemanticGuidedHumanMatting.inference as inference

In [117]:
vid_file = './dataset/shilpa/6.mp4'
id = 'shilpa_6'

vidcap = cv2.VideoCapture(vid_file)
success,image = vidcap.read()
count = 0

frames = './output/frames/'
masks = './output/masks/'

frame_files = glob.glob(frames + '/*')
mask_files = glob.glob(masks + '/*')
for f in frame_files:
    os.remove(f)
for f in mask_files:
    os.remove(f)

while success:
  if(count%4 == 0):
    image = cv2.resize(image, (int(image.shape[1]), int(image.shape[0])))
    # print(int(image.shape[0]/8))
    cv2.imwrite(frames+"%d.jpg" % count, image)  
    # print("Done : %s" %count)   
  success,image = vidcap.read()
  count += 1

In [118]:
# pretrained_weight = './SemanticGuidedHumanMatting/pretrained/SGHM-ResNet50.pth'
# images_dir = './output/frames/'
# gt_dir = ''
# result_dir = './output/masks/'


# if not os.path.exists(pretrained_weight):
#     print('Cannot find the pretrained model: {0}'.format(pretrained_weight))
#     exit()

# model = HumanMatting(backbone='resnet50')
# model = nn.DataParallel(model).eval()
# model.load_state_dict(torch.load(pretrained_weight, map_location=torch.device('cpu')))
# print("Load checkpoint successfully ...")


# image_list = sorted([*glob.glob(os.path.join(images_dir, '**', '*.jpg'), recursive=True),
#                     *glob.glob(os.path.join(images_dir, '**', '*.png'), recursive=True)])

# num_image = len(image_list)
# print("Find ", num_image, " images")

# for i in range(num_image):
#     image_path = image_list[i]
#     image_name = image_path[image_path.rfind('/')+1:image_path.rfind('.')]
#     print(i, '/', num_image, image_name)

#     with Image.open(image_path) as img:
#         img = img.convert("RGB")
#         # img = img

#     pred_alpha, pred_mask = inference.single_inference(model, img)

#     output_dir = result_dir + image_path[len(images_dir):image_path.rfind('/')]
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
#     save_path = output_dir + '/' + os.path.split(image_name)[-1] + '.png'
#     Image.fromarray(((pred_alpha * 255).astype('uint8')), mode='L').save(save_path)


from ultralytics import YOLO


pretrained_weight = './yolov8x-seg.pt'
images_dir = './output/frames/'
result_dir = './output/masks/'

model = YOLO(pretrained_weight)

files = os.listdir('./output/frames/')
for f in files:
    im = cv2.imread('./output/frames/'+'/'+f)
    results = model(im)
    masks = results[0].masks
    if masks:
        ms = masks.data.numpy()
        cv2.imwrite(result_dir + f, ms[0,:,:]*255)


0: 352x640 1 person, 714.8ms
Speed: 1.0ms preprocess, 714.8ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 person, 711.3ms
Speed: 1.0ms preprocess, 711.3ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 person, 689.8ms
Speed: 1.0ms preprocess, 689.8ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 person, 687.6ms
Speed: 1.0ms preprocess, 687.6ms inference, 4.9ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 person, 681.0ms
Speed: 1.0ms preprocess, 681.0ms inference, 5.1ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 person, 1 handbag, 699.2ms
Speed: 1.0ms preprocess, 699.2ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 person, 666.4ms
Speed: 0.0ms preprocess, 666.4ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 1 person, 664.3ms
Speed: 1.0ms preprocess, 664.3ms inference, 4.0ms postprocess p

In [119]:
files = os.listdir('./output/masks/')
thresh = 127

images = []
for f in files:
    im = cv2.imread('./output/masks/'+'/'+f, 0)
    im_bw = cv2.threshold(im, thresh, 255, cv2.THRESH_BINARY)[1]
    images.append(im_bw)

def mass_center(img,is_round=True):
    Y = img.mean(axis=1)
    X = img.mean(axis=0)
    Y_ = np.sum(np.arange(Y.shape[0]) * Y)/np.sum(Y)
    X_ = np.sum(np.arange(X.shape[0]) * X)/np.sum(X)
    if is_round:
        return int(round(X_)),int(round(Y_))
    return X_,Y_

def image_extract(img,newsize):
    if (len(np.where(img.mean(axis=0)!=0)[0]) != 0):
        x_s = np.where(img.mean(axis=0)!=0)[0].min()
        x_e = np.where(img.mean(axis=0)!=0)[0].max()
        
        y_s = np.where(img.mean(axis=1)!=0)[0].min()
        y_e = np.where(img.mean(axis=1)!=0)[0].max()
        
        x_c,_ = mass_center(img)
        x_s = x_c-newsize[1]//2
        x_e = x_c+newsize[1]//2
        img = img[y_s:y_e,x_s if x_s>0 else 0:x_e if x_e<img.shape[1] else img.shape[1]]
        return cv2.resize(img,newsize)
    else:
        return 0

image_data = []
for k in images:
    item = image_extract(k,(64, 128))
    if(np.max(item) != 0):
        image_data.append(item)

gei = np.mean(image_data,axis=0)

cv2.imwrite('./output/gei/'+id+'.png', gei)

plt.imshow(gei)
plt.show()

C:\Users\h4rip\AppData\Local\Temp\ipykernel_20208\2402808924.py:46: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [123]:
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np
import imageio

def aug_gen(img, out_path, img_name):
    ia.seed(1)

    # img = imageio.imread("test.jpg") #read you image
    images = np.array(
        [img for _ in range(32)], dtype=np.uint8)  # 32 means creat 32 enhanced images using following methods.

    seq = iaa.Sequential(
        [            
            iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.5))),        
            iaa.ContrastNormalization((0.75, 1.5)),         
            iaa.AdditiveGaussianNoise(
                loc=0, scale=(0.0, 0.05 * 255), per_channel=0.5),    
        ],
        random_order=True)  # apply augmenters in random order

    images_aug = seq.augment_images(images)

    for i in range(32):
        imageio.imwrite(out_path + str(i)+ '_' + img_name + '.jpg', images_aug[i])  #write all changed images

In [124]:
test = cv2.imread('./output/gei/abhirami/abhirami_0.png')

out_path = './temp/'
img_name = 'abhirami_0'

aug_gen(test, out_path, img_name)

In [142]:
in_path = './fine_tune_dataset/train/shilpa/'
out_folder = 'aug/'

files = os.listdir(in_path)
os.mkdir(in_path + out_folder)

for f in files:
    im = cv2.imread(in_path+f, 0)
    out_path = in_path + out_folder
    img_name = f.split('.')[0]
    aug_gen(im, out_path, img_name)

c:\Users\h4rip\anaconda3\envs\tf\lib\site-packages\imgaug\imgaug.py:184: DeprecationWarning: Function `ContrastNormalization()` is deprecated. Use `imgaug.contrast.LinearContrast` instead.
  warn_deprecated(msg, stacklevel=3)
